## Demonstrate interactions with relations using omnikeeper

In [1]:
import os
import omnikeeper_client as okc
from omnikeeper_client import TraitDefinition, TraitAttributeDefinition, TraitRelationDefinition, ATTRIBUTETYPE_TEXT, ATTRIBUTETYPE_INTEGER
import pandas as pd

As usual create the omnikeeper client

In [2]:
okapiclient = okc.OkApiClient(
    backend_url=os.getenv('OMNIKEEPER_URL'),
    client_id=os.getenv('OMNIKEEPER_AUTH_CLIENTID'),
    username=os.getenv('OMNIKEEPER_AUTH_USERNAME'),
    password=os.getenv('OMNIKEEPER_AUTH_PASSWORD'),
)

Define the trait names and layer

In [3]:
host_trait_name = "python_client_demo.host"
interface_trait_name = "python_client_demo.interface"
layer_name = "relations"

Make sure that traits are created correctly

In [4]:
is_host_trait_created = okc.upsert_trait(okapiclient, TraitDefinition(host_trait_name, [
        TraitAttributeDefinition("id", "host.id", ATTRIBUTETYPE_INTEGER),
      ], optional_relations = [
        TraitRelationDefinition(identifier = "has_interface", predicate_id = "has_interface", direction_forward=True)
      ]))
print(is_host_trait_created)

is_interface_trait_created = okc.upsert_trait(okapiclient, TraitDefinition(interface_trait_name, [
        TraitAttributeDefinition("id", "interface.id", ATTRIBUTETYPE_INTEGER),
      ], optional_relations = [
        TraitRelationDefinition(identifier = "has_host", predicate_id = "has_host", direction_forward=True)
      ]))
print(is_interface_trait_created)

True
True


Create the host and interface CIs

In [5]:
host_ci_id = okc.create_ci(okapiclient, "Host A", layer_name)
host_ci_id

UUID('c05824b5-cb00-4227-aea9-9a73aaa5f909')

Update created host ci, add host.id attribute

In [6]:
host_attributes = {
    "host.id": 1,
}

okc.mutate_ci(okapiclient, layer_name, host_ci_id, host_attributes)

{'mutateCIs': {'affectedCIs': [{'id': 'c05824b5-cb00-4227-aea9-9a73aaa5f909'}]}}

In [7]:
interface_ci_id = okc.create_ci(okapiclient, "Interface A", layer_name)
interface_ci_id

UUID('ed6b1972-97a6-4601-a1fd-d4cc3e52b282')

In [8]:
interface_attributes = {
    "interface.id": 1,
}

okc.mutate_ci(okapiclient, layer_name, interface_ci_id, interface_attributes)

{'mutateCIs': {'affectedCIs': [{'id': 'ed6b1972-97a6-4601-a1fd-d4cc3e52b282'}]}}

Add host and interface relations using the created cis and okc

In [9]:
okc.add_trait_relations_by_ciid(okapiclient, host_trait_name, "has_interface", host_ci_id, [interface_ci_id], layer_name)

True

In [10]:
okc.add_trait_relations_by_ciid(okapiclient, interface_trait_name, "has_host", interface_ci_id, [host_ci_id], layer_name)

True

### Relations operations using lists

Fetch hosts and interfaces from the specified layer

In [11]:
hosts = okc.get_all_traitentities_list(okapiclient, trait_name=host_trait_name, layers=[layer_name])
hosts

[{'ciid': 'c05824b5-cb00-4227-aea9-9a73aaa5f909', 'id': 1}]

In [12]:
interfaces = okc.get_all_traitentities_list(okapiclient, trait_name=interface_trait_name, layers=[layer_name])
interfaces

[{'ciid': 'ed6b1972-97a6-4601-a1fd-d4cc3e52b282', 'id': 1}]

Fetch has_host realtions. In the result interfaces will be as base_cis and hosts as related cis

In [13]:
has_host_relations = okc.get_trait_relation(okapiclient, trait_name=interface_trait_name, relation_name="has_host", layers=[layer_name])
has_host_relations

[{'base_ciid': 'ed6b1972-97a6-4601-a1fd-d4cc3e52b282',
  'related_ciids': ['c05824b5-cb00-4227-aea9-9a73aaa5f909']}]

Fetch has_interface relations for all hosts

In [14]:
has_interface_relations = okc.get_trait_relation(okapiclient, trait_name=host_trait_name, relation_name="has_interface", layers=[layer_name])
has_interface_relations

[{'base_ciid': 'c05824b5-cb00-4227-aea9-9a73aaa5f909',
  'related_ciids': ['ed6b1972-97a6-4601-a1fd-d4cc3e52b282']}]

Merge hosts with realted cis in this case interfaces

In [15]:
interface_dict = {interface['ciid']: interface for interface in interfaces}

hosts_with_relations = []
for host in hosts:
    merged_dict = host.copy()
    merged_dict['related_ciis'] = []
    for relation in has_interface_relations:
        if host['ciid'] == relation['base_ciid']:
            for related_ciid in relation['related_ciids']:
                if related_ciid in interface_dict:
                    merged_dict['related_ciis'].append(interface_dict[related_ciid])
    hosts_with_relations.append(merged_dict)

hosts_with_relations

[{'ciid': 'c05824b5-cb00-4227-aea9-9a73aaa5f909',
  'id': 1,
  'related_ciis': [{'ciid': 'ed6b1972-97a6-4601-a1fd-d4cc3e52b282', 'id': 1}]}]

The same can be done for interfaces to fetch its relations

In [16]:
host_dict = {host['ciid']: host for host in hosts}

interfaces_with_relations = []
for interface in interfaces:
    merged_dict = interface.copy()
    merged_dict['related_ciis'] = []
    for relation in has_host_relations:
        if interface['ciid'] == relation['base_ciid']:
            for related_ciid in relation['related_ciids']:
                if related_ciid in host_dict:
                    merged_dict['related_ciis'].append(host_dict[related_ciid])
    interfaces_with_relations.append(merged_dict)

interfaces_with_relations

[{'ciid': 'ed6b1972-97a6-4601-a1fd-d4cc3e52b282',
  'id': 1,
  'related_ciis': [{'ciid': 'c05824b5-cb00-4227-aea9-9a73aaa5f909', 'id': 1}]}]

### Relations operations using pandas dataframes

In [17]:
# TODO